In [1]:
import pandas as pd
import numpy as np
import glob, os, re, jieba
import time
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.models import Model, save_model, load_model
from keras.layers import Input, LSTM, Dense, Flatten
from keras.callbacks import EarlyStopping
import json

In [2]:
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [3]:
'''gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print('GPU Running')
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        (e)'''

"gpus = tf.config.list_physical_devices('GPU')\nif gpus:\n    try:\n        tf.config.experimental.set_memory_growth(gpus[0], True)\n        print('GPU Running')\n    except RuntimeError as e:\n        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다\n        (e)"

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def preprocess_kr(w):
    w = re.sub(r"([?'!¿\-·\"])", r" \1 ", w)
    w = re.sub(r'[ |ㄱ-ㅎ|ㅏ-ㅣ]+', r" ", w)
    w = re.sub(r"\,(?=[0-9])", r"", w)
    w = w[:-1].strip()
    w = '<start> ' + w + ' <end>'
    return w

In [6]:
def extract_data_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    standard_forms = []
    dialect_forms = []

    for utterance in data['utterance']:
        standard_forms.append(utterance['standard_form'])
        dialect_forms.append(utterance['dialect_form'])

    return standard_forms, dialect_forms

def preprocess(path, num_data):
    files = glob.glob(os.path.join(path, '*.json'))
    std, jej = [], []

    for f in files:
        std_forms, dial_forms = extract_data_from_json(f)
        std.extend(std_forms)
        jej.extend(dial_forms)

    std_series = pd.Series(std)
    jej_series = pd.Series(jej)

    df = pd.concat([std_series, jej_series], axis=1)
    df.columns = ['표준어', '제주어']

    df['표준어'] = df['표준어'].apply(preprocess_kr)
    df['제주어'] = df['제주어'].apply(preprocess_kr)

    df = df.sample(num_data, random_state=2)

    return df


In [7]:
def tokenize(texts):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    tokenizer.fit_on_texts(texts)
    tensor = tokenizer.texts_to_sequences(texts)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, tokenizer

def tokenize_dataset(path, num_data):
    df = preprocess(path, num_data)

    std_tensor, std_tokenizer = tokenize(df['표준어'].values)
    jej_tensor, jej_tokenizer = tokenize(df['제주어'].values)

    return std_tensor, jej_tensor, std_tokenizer, jej_tokenizer

# 설정한 경로와 데이터 크기 제한
num_data = 15000
path = '/content/drive/MyDrive/colab/Jeju/Training'
std_tensor, jej_tensor, std_lang, jej_lang = tokenize_dataset(path, num_data)

max_length_std = std_tensor.shape[1]
max_length_jej = jej_tensor.shape[1]

print('표준어 tensor 최장 길이 : {}'.format(max_length_std))
print('제주어 tensor 최장 길이 : {}'.format(max_length_jej))

std_tensor_train, std_tensor_val, jej_tensor_train, jej_tensor_val = train_test_split(std_tensor, jej_tensor, test_size=0.2)


표준어 tensor 최장 길이 : 34
제주어 tensor 최장 길이 : 34


In [8]:
path

'/content/drive/MyDrive/colab/Jeju/Training'

In [9]:
# 약 5분 소요
std_tensor, jej_tensor, std_lang, jej_lang = tokenize_dataset(path, num_data)

# 입력 텐서와 타겟 텐서의 최대 길이 계산
max_length_std = std_tensor.shape[1]
max_length_jej = jej_tensor.shape[1]

print('표준어 tensor 최장 길이 : {}'.format(max_length_std))
print('제주어 tensor 최장 길이 : {}'.format(max_length_jej))

표준어 tensor 최장 길이 : 34
제주어 tensor 최장 길이 : 34


In [10]:
# 표준어와 제주어 텐서를 훈련 데이터셋과 검증 데이터셋으로 나누기
std_tensor_train, std_tensor_val, jej_tensor_train, jej_tensor_val = train_test_split(std_tensor, jej_tensor, test_size=0.2)

print('훈련 데이터셋 크기 : {}, {}'.format(len(std_tensor_train), len(jej_tensor_train)))
print('검증 데이터셋 크기 : {}, {}'.format(len(std_tensor_val), len(jej_tensor_val)))

훈련 데이터셋 크기 : 12000, 12000
검증 데이터셋 크기 : 3000, 3000


In [11]:
def convert(tokenizer, tensor):
    for t in tensor:
        if t != 0:
            print("%12d ----> %s" % (t, tokenizer.index_word[t]))

print('표준어 index ----> token')
convert(std_lang, std_tensor_train[0])
print()
print('제주어 index ----> token')
convert(jej_lang, jej_tensor_train[0])


표준어 index ----> token
           1 ----> <start>
          37 ----> 응
       22528 ----> 초를
         409 ----> 처음에
       22529 ----> 쓰던
       22530 ----> 초는
          18 ----> 그렇게
           8 ----> 안
         160 ----> 했는
           2 ----> <end>

제주어 index ----> token
           1 ----> <start>
          36 ----> 응
       24174 ----> 초를
         400 ----> 처음에
       24175 ----> 쓰던
       24176 ----> 초는
          95 ----> 경
           8 ----> 안
         322 ----> 해신
           2 ----> <end>


In [12]:
BUFFER_SIZE = len(std_tensor_train)
BATCH_SIZE = 128  # Out of Memory 에러 주의
steps_per_epoch = len(std_tensor_train) // BATCH_SIZE
embedding_size = 1024
units = 1024

vocab_input_size = len(std_lang.word_index) + 1
vocab_target_size = len(jej_lang.word_index) + 1

print('표준어 토큰 개수 : {}'.format(vocab_input_size))
print('제주어 토큰 개수 : {}'.format(vocab_target_size))


표준어 토큰 개수 : 23368
제주어 토큰 개수 : 25094


In [13]:
dataset = tf.data.Dataset.from_tensor_slices((std_tensor_train, jej_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [14]:
example_input_batch, example_target_batch = next(iter(dataset))

print(example_input_batch.shape, example_target_batch.shape)

(128, 34) (128, 34)


In [15]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_size, self.enc_units))

In [16]:
encoder = Encoder(vocab_input_size, embedding_size, units, BATCH_SIZE)

#샘플 입력
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)

print(f'Encoder output (batch size, sequence length, units) = {sample_output.shape}')
print(f'Encoder Hidden state  (batch size, units) = {sample_hidden.shape}')

Encoder output (batch size, sequence length, units) = (128, 34, 1024)
Encoder Hidden state  (batch size, units) = (128, 1024)


In [17]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        #query hidden state는 (batch_size, hidden_size)로 구성
        #query_with_time_axis는 (batch_size, 1, hidden_size)로 구성
        #values는 (batch_size, max_len, hidden_size)로 구성
        query_with_time_axis = tf.expand_dims(query, 1)

        #score는 (batch_size, max_len, units)로 구성
        #score를 self.V에 적용하기 때문에 마지막 축에 1을 얻어 (batch_size, max_len, 1)로 구성되게 됨
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(query_with_time_axis)))

        #attention_weights는 (batch_size, max_len, 1)로 구성
        attention_weights = tf.nn.softmax(score, axis=1)

        #병합 이후 context_vector는 (batch_size, hidden_size)로 구성
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [18]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print(f"Attention result shape: (batch size, units) {attention_result.shape}")
print(f"Attention weights shape: (batch_size, sequence_length, 1) {attention_weights.shape}")

Attention result shape: (batch size, units) (128, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (128, 34, 1)


In [19]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_size):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):    # 단어 하나하나 해석 진행
        #            hidden (batch_size, units),    enc_output (batch_size, max_length_inp, enc_units)
        # =>context_vector (batch_size, enc_units), attention_weights (batch_size, max_length_inp, 1)
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # 임베딩 층 통과 후 x는 (batch_size, 1, embedding_dim)로 구성
        x = self.embedding(x)

        #context vector과 임베딩 결과를 결합한 후 x는 (batch_size, 1, embedding_dim+hidden_size)로 구성
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        #위에서 결합된 벡터를 GRU에 전달
        output, state = self.gru(x)

        #output은 (batch_size*1, hidden_size)로 구성
        output = tf.reshape(output, (-1, output.shape[2]))

        #FC(완전연결층)을 지난 x는 (batch_size, vocab)으로 구성
        x = self.fc(output)

        return x, state, attention_weights

In [20]:
decoder = Decoder(vocab_target_size, embedding_size, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)), sample_hidden, sample_output)

print(f'Decoder output shape: (batch_size, vocab size) {sample_decoder_output.shape}')

Decoder output shape: (batch_size, vocab size) (128, 25094)


In [21]:
optimizer = tf.keras.optimizers.Adam()
loss_objects = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                             reduction='none')


def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_objects(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [22]:
#체크포인트(객체 기반 저장)
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "cpkt")

checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

manager = tf.train.CheckpointManager(checkpoint, directory=checkpoint_dir,
                                     checkpoint_name='model.ckpt',
                                     max_to_keep=3)

In [23]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([jej_lang.word_index['<start>']] * BATCH_SIZE, 1)

        for t in range(1, targ.shape[1]):
            # enc_output을 디코더에 전달
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)  # teacher forcing

    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss


In [ ]:

EPOCHS =90

for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 100 == 0:
            print("Epoch {} Batch {} Loss {:.4f}".format(epoch+1, batch, batch_loss.numpy()))

    print('Epoch {} Loss {:.4f}'.format(epoch+1, total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

    checkpoint.save(file_prefix=checkpoint_prefix)
    manager.save()


Epoch 1 Batch 0 Loss 1.7366


In [ ]:
def evaluate(sentence):
    attention_plot = np.zeros((max_length_jej, max_length_std))

    sentence = preprocess_kr(sentence)

    inputs = [std_lang.word_index.get(i, 0) for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_std, padding='post')

    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([jej_lang.word_index['<start>']], 0)

    for t in range(max_length_jej):
        predictions, dec_hidden, attention_weights = decoder(
            dec_input, dec_hidden, enc_out)

        # 나중에 attention 가중치를 시각화하기 위해 저장해두기
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += jej_lang.index_word.get(predicted_id, '') + ' '

        if jej_lang.index_word.get(predicted_id) == '<end>':
            return result, sentence, attention_plot

        # 예측된 id를 모델에 다시 feeding
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(20, 20))
    ax = fig.add_subplot(1, 1, 1)
    cax = ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 16}
    ax.set_xticklabels([''] + sentence.split(' '), fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence.split(' '), fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    fig.colorbar(cax)
    plt.show()


In [ ]:
def translate(sentence):
    result, sentence, attention_plot = evaluate(sentence)

    print("Input : %s" % (sentence))
    print("Translation : {}".format(result))



In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
val_df = pd.read_csv('/content/drive/MyDrive/colab/Jeju/val_df.csv', index_col=0)
val_sample = val_df.sample(10)
val_sample

In [ ]:
for idx, i in enumerate(val_sample['표준어'].values):
    try:
        translate(u'{}'.format(i))
        print("Intended Output : %s" % (val_sample.iloc[idx,1]))
        print(" ")
    except:
        print(i, '=> 데이터셋에 없는 단어 포함')
        print(" ")